# Data Science Course
### &nbsp; &nbsp; &nbsp; Michaël Defferrard, Sept. 2016
## Lecture 2: A Python Tour of Data Science

The goal of this short primer is to introduce the Python stack for Data Science. It is designed as a tour around the major Python packages used for the main computational tasks encountered in Data Science. Before starting, two notes:
1. There exists faster / better ways to accomplish the presented computations. The goal is to present the packages.
1. It is not meant to teach you (scientific) Python. I however tried to include the main construtions and idioms of the language and packages.

A typical Data Science **pipeline**:
1. Data acquisition
    1. Importation
    1. Cleaning
    1. Exploration
1. Data exploitation
    1. Information extraction
    1. Prediction

A **motivating example**: predict whether a credit card client will default.
* Binary classification task: client will default or not ($y=1$ if yes; $y=0$ if no).
* 30'000 clients from Taiwan.
* 23 numerical & categorical explanatory variables:
    1. $x_1$: Amount of the given credit.
    2. $x_2$: Gender (1 = male; 2 = female).
    3. $x_3$: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others).
    4. $x_4$: Marital status (1 = married; 2 = single; 3 = others).
    5. $x_5$: Age (year).
    6. $x_6$ to $x_{11}$: History of past payment (monthly from September to April, 2005) (-1 = pay duly; 1 = payment delay for one month; ...; 9 = payment delay for nine months and above).
    7. $x_{12}$ to $x_{17}$: Amount of bill statement (monthly from September to April, 2005).
    8. $x_{18}$ to $x_{23}$: Amount of previous payment (monthly from September to April, 2005).
* Found on the [UCI ML repository](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients).

In [ ]:
import numpy as np
import sympy as sp
import pandas as pd
import sqlalchemy
from sklearn import linear_model, metrics
from matplotlib import pyplot as plt
plt.style.use('ggplot')

import time

# Or notebook for interaction.
%matplotlib inline

## 1 Data acquisition

* The world is messy: we got data from CSV, JSON, Excel, HDF5, SQL database.
* Could also be: matlab, HTML/XML, web scraping, web APIs (e.g. Twitter Firhose), noSQL databases.

In [ ]:
%%bash
ls data

### 1.1 Importing from a database

[SQLAlchemy](http://www.sqlalchemy.org/) to the rescue.
* Abstraction between DBAPIs.
    * Supported databases: SQLite, Postgresql, MySQL, Oracle, MS-SQL, Firebird, Sybase and others.
* [SQL Expression Language](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html).
* [Object Relational Mapper (ORM)](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html).

TODO: show some ORM

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///data/payments.sqlite', echo=False)

# Infer from existing DB.
metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

# An SQL SELECT statement.
table = metadata.tables.get('payments')
op = sqlalchemy.sql.select([table])
engine.echo = True
result = engine.execute(op)
engine.echo = False

In [ ]:
# Show some lines.
for row in result.fetchmany(size=10):
    print('ID: {:2d}, payments: {}'.format(row[0], row[1:]))
result.close()

In [ ]:
# Some raw SQL.
paid = 1000
op = sqlalchemy.sql.text('SELECT payments."ID", payments."PAY6" FROM payments WHERE payments."PAY6" = {}'.format(paid))
result = engine.execute(op).fetchall()
print('{} clients paid {} in April 2005'.format(len(result), paid))

### 1.2 Merging data sources

Put some [pandas](http://pandas.pydata.org/) in our Python !
* Import / export data from / to various sources.
* Data frames manipulations: slicing, dicing, grouping.
* And many more !

In [ ]:
def get_data(folder):
    demographics = pd.read_csv(folder + 'demographics.csv', skiprows=[0], index_col=0)
    delays = pd.read_excel(folder + 'delays.xls', skiprows=[0], index_col=0)
    bills = pd.read_hdf(folder + 'bills.hdf5', 'bills')
    payments = pd.read_sql('payments', engine, index_col='ID')

    target = pd.read_json(folder + 'target.json', typ='series', orient='index')
    target = pd.DataFrame(target, columns=['DEFAULT'])

    return pd.concat([demographics, delays, bills, payments, target], axis=1)

data = get_data('data/')
attributes = data.columns.tolist()

# Tansform from numerical to categorical variable.
data['SEX'] = data['SEX'].astype('category')
data['SEX'].cat.categories = ['MALE', 'FEMALE']
data['MARRIAGE'] = data['MARRIAGE'].astype('category')
data['MARRIAGE'].cat.categories = ['UNK', 'MARRIED', 'SINGLE', 'OTHERS']
data['EDUCATION'] = data['EDUCATION'].astype('category')
data['EDUCATION'].cat.categories = ['UNK', 'GRAD SCHOOL', 'UNIVERSITY', 'HIGH SCHOOL', 'OTHERS', 'UNK1', 'UNK2']

### 1.3 Looking at the data

In [ ]:
data.loc[:6, ['LIMIT', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'DEFAULT']]

In [ ]:
data.iloc[:5, 4:10]

In [ ]:
data.iloc[:5, 11:23]

Export as an [HTML table](./subset.html) for manual inspection.

In [ ]:
data[:1000].to_html('subset.html')

## 2 Data cleaning

The most boring part. But the most time intensive !

TODO: show a study. Which one was it, KDnuggets ?

Marital status
* Should be either 1 (married), 2 (single) or 3 (others).
* Let's *assume* the 0 represents errors when collecting the data and remove those clients.

In [ ]:
print(data['MARRIAGE'].value_counts())
data = data[data['MARRIAGE'] != 'UNK']
data['MARRIAGE'] = data['MARRIAGE'].cat.remove_unused_categories()
print('\nWe are left with {} clients\n'.format(data.shape))
print(data['MARRIAGE'].unique())

Education
* Should be either 1 (graduate school), 2 (university), 3 (high school) or 4 (others).
* Let's *assume* the 0, 5, 6 are dubious, but do not invalidate the data. Keep them as they may have predictive power.

In [ ]:
print(data['EDUCATION'].value_counts())
data.loc[data['EDUCATION']=='UNK1', 'EDUCATION'] = 'UNK'
data.loc[data['EDUCATION']=='UNK2', 'EDUCATION'] = 'UNK'
data['EDUCATION'] = data['EDUCATION'].cat.remove_unused_categories()
print(data['EDUCATION'].value_counts())

## 3 Data exploration

* Get descriptive statistics.
* Plot informative figures.
* Verify some intuitive correlations.

TODO: further exploration with [statsmodels](http://statsmodels.sourceforge.net/)

In [ ]:
attributes_numerical = ['LIMIT', 'AGE']
attributes_numerical.extend(attributes[11:23])
data.loc[:, attributes_numerical].describe().astype(np.int)

In [ ]:
data.loc[:, 'AGE'].plot.hist(bins=20, figsize=(15,5))
ax = data.iloc[:, 11:17].plot.box(logy=True, figsize=(15,5))

In [ ]:
percentage = data[data.DEFAULT == 1].shape[0] / data.shape[0] * 100
print('Percentage of defaults: {:.2f}%'.format(percentage))

Who's more susceptible to default, males or females ?
Statistical signifiance could be tested with [scipy.stats](http://docs.scipy.org/doc/scipy/reference/stats.html).

In [ ]:
pd.crosstab(data['SEX'], data['DEFAULT'])

**Intuition**: people who pay late present a higher risk of defaulting. Let's verify ! 

In [ ]:
group = data.groupby('DELAY1').mean()
group['DEFAULT'].plot(grid=True, figsize=(15,5));

## 4 Pre-processing

Back to [NumPy](http://www.numpy.org/), the fundamental package for scientific computing with Python. It provides multi-dimensional arrays, data types and linear algebra routines.

1. Transform data types.
1. Data normalization.
    * Some algorithms expect data to be centered and scaled.
    * Some will train faster.
1. Train / test splitting.

In [ ]:
# Back to numeric values.
# Note: in a serious project, these should be treated as categories.
data['SEX'].cat.categories = [-1, 1]
data['SEX'] = data['SEX'].astype(np.int)
data['MARRIAGE'].cat.categories = [-1, 1, 0]
data['MARRIAGE'] = data['MARRIAGE'].astype(np.int)
data['EDUCATION'].cat.categories = [-2, 2, 1, 0, -1]
data['EDUCATION'] = data['EDUCATION'].astype(np.int)

data['DEFAULT'] = data['DEFAULT'] * 2 - 1  # [0,1] --> [-1,1]

In [ ]:
# Observations and targets.
X = data.values[:,:23]
y = data.values[:,23]
n, d = X.shape
print('The data is a {} with {} samples of dimensionality {}.'.format(type(X), n, d))

In [ ]:
# Center and scale.
# Note: on a serious project, should be done after train / test split.
X = X.astype(np.float)
X -= X.mean(axis=0)
X /= X.std(axis=0)

In [ ]:
# Training and testing sets.
test_size = 10000
print('Split: {} testing and {} training samples'.format(test_size, y.size - test_size))
perm = np.random.permutation(y.size)
X_test  = X[perm[:test_size]]
X_train = X[perm[test_size:]]
y_test  = y[perm[:test_size]]
y_train = y[perm[test_size:]]

## 5 Visualization

[matplotlib](http://matplotlib.org/) is the goto 2D plotting library.

TODO: interactive (widget, [Bokeh](http://bokeh.pydata.org))

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(121)
plt.plot(data.values[:,4],'.')
plt.title('Original')
plt.xlabel('Sample')
plt.ylabel('Value')

plt.subplot(122)
plt.plot(X[:,4],'.')
plt.title('Centered and scaled')
plt.xlabel('Sample')
plt.ylabel('Value')
plt.show()

## 6 A first predictive model

The ingredients of a Machine Learning (ML) model:
1. Predictive function: $f(x) = xw^T + b$ (a linear transformation)
1. Error function: $E = \sum_{i=1}^n \left( f(x_i) - y_i \right)^2 = \| f(X) - y \|_2^2$ (least squares)
1. Regularization (optional): $R = \|w\|_2^2$ (Thikonov)
1. Loss / objective function: $L = E + \alpha R$

Our model has a sole hyper-parameter, $\alpha \geq 0$, which controls the shrinkage

A Machine Learning (ML) problem can often be cast as a (convex or smooth) optimization problem:

1. Optimization problem
$$\hat{w} = \operatorname*{arg min}_w \| Xw + b - y \|_2^2 + \alpha \|w\|_2^2$$
1. Gradient
$$\frac{\partial L}{\partial{w}} = 2 X^T (Xw+b-y) + 2\alpha w$$
$$\frac{\partial L}{\partial{b}} = 2 \sum_{i=1}^n (x_iw+b-y_i) = 2 \sum_{i=1}^n (x_iw-y_i) + 2n \cdot b$$
1. Closed-form solution:
$$\frac{\partial L}{\partial{w}} = 0 \ \rightarrow \ 2 X^T X\hat{w} + 2\alpha \hat{w} = 2 X^T y - 2 X^T b \ \rightarrow \ \hat{w} = (X^T X + \alpha I)^{-1} X^T (y-b)$$
$$\frac{\partial L}{\partial{b}} = 0 \ \rightarrow \ 2n\hat{b} = 2\sum_{i=1}^n (y_i) - \underbrace{2\sum_{i=1}^n (x_iw)}_{=0} \ \rightarrow \ \hat{b} = \frac1n I^T y$$

What if the resulting problem is non-smooth ? See the [PyUNLocBoX](http://pyunlocbox.readthedocs.io), a convex optimization toolbox which implements [proximal splitting methods](https://en.wikipedia.org/wiki/Proximal_gradient_method).

### 5.1 Take a *symbolic* derivative

[SymPy](http://www.sympy.org/) is our computer algebra system (CAS) of choice.

In [ ]:
X, y, w, b, a = sp.symbols('x y w b a')
L = (X*w + b - y)**2 + a*w**2
dLdw = sp.diff(L, w)
dLdb = sp.diff(L, b)
print('L = {}'.format(L))
print('  dL/dw = {}'.format(dLdw))
print('  dL/db = {}'.format(dLdb))

### 5.2 Build the classifier

We solely use the [NumPy](http://www.numpy.org/) linear algebra capabilities.

In [ ]:
class ridge_regression(object):
    """Our ML model."""
    
    def __init__(self, alpha=0):
        "The model's constructor. Initialize the hyper-parameters."
        self.a = alpha
    
    def predict(self, X):
        """Return the predicted class given the features."""
        return np.sign(X.dot(self.w) + self.b)
    
    def fit(self, X, y):
        """Learn the model's parameters given the training data, the closed-form way."""
        n, d = X.shape
        self.b = np.mean(y)
        Ainv = np.linalg.inv(X.T.dot(X) + self.a * np.identity(d))
        self.w = Ainv.dot(X.T).dot(y - self.b)

    def loss(self, X, y, w=None, b=None):
        w = self.w if w is None else w
        b = self.b if b is None else b
        return np.linalg.norm(X.dot(w) + b - y) + self.a * np.linalg.norm(w, 2)

In [ ]:
def accuracy(y_pred, y_true):
    """Our evaluation metric, the classification accuracy."""
    return np.sum(y_pred == y_true) / y_true.size

def evaluate(model):
    t = time.process_time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy(y_pred, y_test)
    loss = model.loss(X_test, y_test)
    t = time.process_time() - t
    print('accuracy: {:.2f}%, loss: {:.2f}, time: {:.2f}ms'.format(acc*100, loss, t*1000))

alpha = 1e-2*n
evaluate(ridge_regression(alpha))

In [ ]:
def fit_lapack(self, X, y):
    """Better way (numerical stability): solve the linear system with LAPACK."""
    n, d = X.shape
    self.b = np.mean(y)
    A = X.T.dot(X) + self.a * np.identity(d)
    b = X.T.dot(y - self.b)
    self.w = np.linalg.solve(A, b)

# Observe that Python is a dynamic language.
ridge_regression.fit = fit_lapack

evaluate(ridge_regression(alpha))

### 5.3 Learning as optimization

Feel the power of [SciPy](https://www.scipy.org/) ! It provides higher-level algorithms, e.g. optimization, interpolation, signal processing, sparse matrices, decompositions.

In [ ]:
class ridge_regression_optimize(ridge_regression):
    
    def __init__(self, alpha=0, method=None):
        super().__init__(alpha)
        self.method = method
    
    def fit(self, X, y):
        """Fitted with a general purpose optimization algorithm."""
        from scipy.optimize import minimize
        n, d = X.shape
        self.b = np.mean(y)
        
        f = lambda w: self.loss(X, y, w)
        
        def jac(w):
            A = X.dot(w) + self.b - y
            return 2 * X.T.dot(A) + 2 * self.a * w
            
        w0 = np.random.normal(size=d)
        res = minimize(f, w0, method=self.method, jac=jac)
        self.w = res.x

#evaluate(ridge_regression_optimize(alpha=1e-2*n))
evaluate(ridge_regression_optimize(alpha, method='Nelder-Mead'))
evaluate(ridge_regression_optimize(alpha, method='BFGS'))

### 5.4 Automatic differentiation

[autograd](https://github.com/HIPS/autograd/) is our tool of choice for [automatic differentation](https://en.wikipedia.org/wiki/Automatic_differentiation).

## 6 ML models made easier

Tired of writing algorithms ? Try [scikit-learn](http://scikit-learn.org), which provides many tools for data mining and analysis.

In [ ]:
model = linear_model.Ridge(alpha)
model.fit(X_train, y_train)
y_pred = np.sign(model.predict(X_test))
acc = metrics.accuracy_score(y_test, y_pred)
print('accuracy: {:.2f}%'.format(acc*100))

In [ ]:
model = linear_model.LogisticRegression()
model.fit(X_train, y_train)
acc = model.score(X_test, y_test)
print('accuracy: {:.2f}%'.format(acc*100))

## 7 Deep Learning

Two Python low-level libraries:
1. [TensorFlow](https://www.tensorflow.org/)
1. [Theano](http://deeplearning.net/software/theano/)

Higher-level libraries:
1. [Keras](https://keras.io/): TensorFlow & Theano backends
1. [Lasagne](http://lasagne.readthedocs.io): Theano backend
1. [Blocks](http://blocks.readthedocs.io): Theano backend
1. [TFLearn](http://tflearn.org): TensorFlow backend

In [ ]:
import keras

model = keras.models.Sequential()
model.add(keras.layers.Dense(output_dim=46, input_dim=23, activation='relu'))
model.add(keras.layers.Dense(output_dim=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train/2+0.5, nb_epoch=5, batch_size=32)

classes = model.predict_classes(X_test, batch_size=32)
proba = model.predict_proba(X_test, batch_size=32)
loss_acc = model.evaluate(X_test, y_test/2+0.5, batch_size=32)

print('\n\nTesting set: {}'.format(loss_acc))

## 9 Is Python slow ?

As Python is an interpreted language, its executation can be slow compared to compiled languages.
We have two ways around it:
1. Compile it to machine code. [Numba](http://numba.pydata.org) is our Just-in-Time (JIT) compiler of choice. An alternative would be [PyPy](http://pypy.org), but it does not support NumPy and lacks behind Cython. Or [Jython](http://www.jython.org) which runs Python on the Java platform, but is Python 2 only.
1. Use specialized wrapper libraries. E.g. NumPy gives a wrapper to BLAS / LAPACK implementations.

In [ ]:
def accuracy_numpy(y_pred, y_true):
    """Using NumPy, implemented in C."""
    return accuracy(y_pred, y_true)

def accuracy_python(y_pred, y_true):
    """Plain Python implementation."""
    num_total = 0
    num_correct = 0
    for y_pred_i, y_true_i in zip(y_pred, y_true):
        num_total += 1
        if y_pred_i == y_true_i:
            num_correct += 1
    return num_correct / num_total
    
from numba import jit
# Decorator, same as accuracy_numba = jit(accuracy_python)

@jit
def accuracy_numba(y_pred, y_true):
    """Plain Python implementation, compiled by LLVM through Numba."""
    num_total = 0
    num_correct = 0
    for y_pred_i, y_true_i in zip(y_pred, y_true):
        num_total += 1
        if y_pred_i == y_true_i:
            num_correct += 1
    return num_correct / num_total

%timeit accuracy_numpy(y_test, y_test)
%timeit accuracy_python(y_test, y_test)
%timeit accuracy_numba(y_test, y_test)